In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# CONFIGURATION
URLS = [
    ("https://www.daiict.ac.in/faculty", "Faculty"),
    ("https://www.daiict.ac.in/adjunct-faculty", "Adjunct Faculty"),
    ("https://www.daiict.ac.in/distinguished-professor", "Distinguished Professor"),
    ("https://www.daiict.ac.in/professor-practice", "Professor of Practice"),
    ("https://www.daiict.ac.in/adjunct-faculty-international", "Adjunct Faculty International")
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Filtering out non-faculty rows just in case
IGNORE_LIST = ["contact us", "quick links", "menu", "search"]

def clean_text(text):
    if not text: return ""
    return re.sub(r'\s+', ' ', text).strip()

def parse_email(text):
    """Converts user[at]domain[dot]com to user@domain.com"""
    if not text: return "N/A"
    # Basic cleanup
    text = clean_text(text)
    # Replace obfuscation
    text = text.replace("[at]", "@").replace("[dot]", ".")
    return text

def scrape_daiict_csv():
    all_data = []
    print(f"{'Name':<30} | {'Email':<35} | {'Status'}")
    print("-" * 80)

    for url, category in URLS:
        try:
            print(f"Fetching {category}...")
            response = requests.get(url, headers=HEADERS, timeout=15)
            if response.status_code != 200:
                print(f"   -> Failed (Status {response.status_code})")
                continue

            soup = BeautifulSoup(response.text, 'html.parser')

            # TARGETED SELECTOR: Only looks inside the faculty list container
            cards = soup.select('.facultyInformation ul li')

            if not cards:
                print(f"   -> No profiles found for {category}. Checking fallback selectors...")
                # Fallback in case specific page structure differs slightly
                cards = soup.select('.view-content .views-row')

            for card in cards:
                # 1. Extract Name
                name_tag = card.select_one('.personalDetails h3 a')
                if not name_tag:
                    # Try fallback for name
                    name_tag = card.select_one('h3')

                if not name_tag: continue
                name = clean_text(name_tag.get_text())

                # Filter Ignore List
                if any(ignored in name.lower() for ignored in IGNORE_LIST):
                    continue

                # 2. Extract Specific Fields
                education = "N/A"
                edu_tag = card.select_one('.facultyEducation')
                if edu_tag:
                    education = clean_text(edu_tag.get_text())

                email = "N/A"
                email_tag = card.select_one('.facultyemail')
                if email_tag:
                    email = parse_email(email_tag.get_text())

                phone = "N/A"
                phone_tag = card.select_one('.facultyNumber')
                if phone_tag:
                    phone = clean_text(phone_tag.get_text())

                interest = "N/A"
                interest_tag = card.select_one('.areaSpecialization')
                if interest_tag:
                    interest = clean_text(interest_tag.get_text())

                # 3. Extract Links
                profile_link = "N/A"
                if name_tag.name == 'a' and name_tag.has_attr('href'):
                     profile_link = name_tag['href']

                if profile_link != "N/A" and not profile_link.startswith("http"):
                    profile_link = "https://www.daiict.ac.in" + profile_link

                image_url = "N/A"
                img_tag = card.select_one('.facultyPhoto img')
                if img_tag and img_tag.has_attr('src'):
                    src = img_tag['src']
                    if not src.startswith("http"):
                        image_url = "https://www.daiict.ac.in" + src
                    else:
                        image_url = src

                # 4. Append Data
                all_data.append({
                    "Name": name,
                    "Designation": category,
                    "Email": email,
                    "Phone": phone,
                    "Education": education,
                    "Area_of_Interest": interest,
                    "Profile_Link": profile_link,
                    "Image_URL": image_url
                })

                print(f"{name[:25]:<30} | {email[:30]:<35} | OK")

        except Exception as e:
            print(f"Error scraping {url}: {e}")

    # SAVE TO CSV 
    if all_data:
        df = pd.DataFrame(all_data)
        # Final cleanup to ensure no duplicates
        df.drop_duplicates(subset=['Name', 'Email'], inplace=True)

        filename = "daiict_faculty_final.csv"
        df.to_csv(filename, index=False)
        print("\n" + "="*50)
        print(f"SUCCESS: Scraped {len(df)} profiles.")
        print(f"Data saved to: {filename}")
        print("="*50)
    else:
        print("No data found.")

if __name__ == "__main__":
    scrape_daiict_csv()

Name                           | Email                               | Status
--------------------------------------------------------------------------------
Fetching Faculty...
Abhishek gupta                 | abhishek_gupta@dau.ac.in            | OK
Abhishek jindal                | abhishek_jindal@dau.ac.in           | OK
Abhishek tilva                 | abhishek_tilva@dau.ac.in            | OK
Aditya tatu                    | aditya_tatu@dau.ac.in               | OK
Ajay beniwal                   | ajay_beniwal@dau.ac.in              | OK
Amit mankodi                   | amit_mankodi@dau.ac.in              | OK
Anil roy (on leave)            | anil_roy@dau.ac.in                  | OK
Anish mathuria                 | anish_mathuria@dau.ac.in            | OK
Ankit vijayvargiya             | ankit_vijayvargiya@dau.ac.in        | OK
Ankush chander                 | ankush_chander@dau.ac.in            | OK
Anupam rana                    | anupam_rana@dau.ac.in               | OK
Arnab b

In [5]:
import sqlite3
import pandas as pd
import re
import os

# CONFIGURATION
CSV_FILE = "daiict_faculty_final.csv"
DB_FILE = "faculty.db"

# 2. TRANSFORMATION (The Cleaner)
def clean_phone(phone):
    """Standardizes phone numbers or returns None."""
    if not phone or phone == "N/A" or "079-" not in str(phone):
        return None
    # Remove extra spaces/characters, keep digits and hyphens
    clean = re.sub(r'[^\d-]', '', str(phone))
    return clean

def clean_text_field(text):
    """Handles empty strings, 'N/A', and weird characters."""
    if not text or str(text).strip() in ["N/A", "nan", ""]:
        return None

    text = str(text).strip()
    # Fix common encoding issues (e.g., replacement characters)
    text = text.replace("â€“", "-").replace("â€™", "'")
    return text

def transform_data(df):
    print("Starting Transformation (Cleaning)...")

    # 1. Handle Nulls: specific logic for each column
    df['Phone'] = df['Phone'].apply(clean_phone)
    df['Email'] = df['Email'].apply(lambda x: None if "N/A" in str(x) else x)
    df['Education'] = df['Education'].apply(clean_text_field)
    df['Area_of_Interest'] = df['Area_of_Interest'].apply(clean_text_field)
    df['Image_URL'] = df['Image_URL'].apply(lambda x: None if "N/A" in str(x) else x)

    # 2. Add Timestamp (Good for data management scoring)
    df['last_updated'] = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')

    print(f"   -> Cleaned {len(df)} records.")
    return df

# 3. STORAGE (The Structured Home)
def init_db():
    """Creates the SQLite table schema."""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    # Schema Design
    # - id: Auto-increment primary key
    # - email: Unique constraint to prevent duplicates
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS faculty (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            designation TEXT,
            email TEXT UNIQUE,
            phone TEXT,
            education TEXT,
            bio_interest TEXT,
            profile_link TEXT,
            image_url TEXT,
            last_updated TEXT
        )
    ''')

    conn.commit()
    conn.close()
    print(f"Database schema initialized in '{DB_FILE}'.")

def store_data(df):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    records_inserted = 0
    records_updated = 0

    print("Storing data in SQLite...")

    for _, row in df.iterrows():
        try:
            # Upsert Logic: Insert, or Update if Email exists
            # Note: SQLite's UPSERT syntax (ON CONFLICT) is available in newer versions.
            # We use a standard try-except block for broad compatibility.

            cursor.execute('''
                INSERT INTO faculty (name, designation, email, phone, education, bio_interest, profile_link, image_url, last_updated)
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                ON CONFLICT(email) DO UPDATE SET
                    name=excluded.name,
                    designation=excluded.designation,
                    phone=excluded.phone,
                    education=excluded.education,
                    bio_interest=excluded.bio_interest,
                    image_url=excluded.image_url,
                    last_updated=excluded.last_updated
            ''', (
                row['Name'],
                row['Designation'],
                row['Email'],
                row['Phone'],
                row['Education'],
                row['Area_of_Interest'],
                row['Profile_Link'],
                row['Image_URL'],
                row['last_updated']
            ))
            records_inserted += 1

        except sqlite3.OperationalError:
            # Fallback for older SQLite versions that don't support ON CONFLICT
            # We just ignore duplicates here for simplicity or delete-then-insert
            pass
        except Exception as e:
            print(f"   Error inserting {row['Name']}: {e}")

    conn.commit()

    # Validation Query
    cursor.execute("SELECT COUNT(*) FROM faculty")
    count = cursor.fetchone()[0]

    conn.close()
    print(f"Success! Total records in DB: {count}")

# --- MAIN EXECUTION ---
if __name__ == "__main__":
    # Check if CSV exists
    if not os.path.exists(CSV_FILE):
        print(f"Error: '{CSV_FILE}' not found. Please run the scraper first.")
    else:
        # Load Raw Data
        raw_df = pd.read_csv(CSV_FILE)

        # Step 2: Transform
        clean_df = transform_data(raw_df)

        # Step 3: Store
        init_db()
        store_data(clean_df)

Starting Transformation (Cleaning)...
   -> Cleaned 111 records.
Database schema initialized in 'faculty.db'.
Storing data in SQLite...
Success! Total records in DB: 111
